# 📌 Estrutura do Notebook 02A-Ingestao-Bronze-TodasPartidas
📍 Objetivo: Ler os dados da camada Bronze, aplicar transformações e salvar na Silver.

In [0]:
%run "/Users/caio.santos.cavalheiro@gmail.com/00-Configuracao"

## 🔶 Leitura dos dados da camada Bronze 

In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession

# Definir o caminho do arquivo CSV na Bronze
csv_file_path = f"s3a://{aws_bucket_name}/bronze/BrasilSerieA_2024_TodasPartidas.csv"

# Ler o arquivo CSV da camada Bronze
df_bronze = spark.read.csv(csv_file_path, header=True, inferSchema=True, sep=";")

print(f'✅ Leitura do Arquivo BrasilSerieA_2024_TodasPartidas na camada Bronze feita com sucesso ')

✅ Leitura do Arquivo BrasilSerieA_2024_TodasPartidas na camada Bronze feita com sucesso 


Leitura das credenciais de forma segura pelo Spark 🔐


## 🔶 Aplicar transformações (limpeza e padronização) 

Limpezas que serão aplicadas no arquivo `BrasilSerieA_2024_TodasPartidas.csv`



- 1️⃣. Remoção de colunas que não serão utilizadas (Colunas com dados de ODDs)
- 2️⃣. Filtragem da Temporada [`Season`] igual a **2024**
- 3️⃣. Remoção das linhas duplicadas
- 4️⃣. Remoção de valores vazios em todas as colunas
- 5️⃣. Remoção de valores vazios em linhas que não tenham valor em `Home`, `Away` ou `Res`
- 6️⃣. Criar `Match_ID` (Identificador único da partida)
- 7️⃣. Criar `Trismestre` que será usado posteriormente
- 8️⃣. Criar a coluna `Turno`, para identificarmos os jogos do Primeiro e Segundo Turno


In [0]:

from pyspark.sql.functions import col, lit, concat_ws, monotonically_increasing_id, quarter, when, to_date, sum

# Removendo colunas de ODDs que não entrarão na análise
df_silver  = df_bronze[['Country', 'League', 'Season', 'Date', 'Time', 'Home', 'Away', 'HG', 'AG', 'Res']]

In [0]:
# Somente partidas com 'Date = 2024'
df_silver = df_silver.filter(col("Season") == 2024)

In [0]:
# 3️⃣. Remover duplicatas
df_silver = df_silver.dropDuplicates()

# 4️⃣. Remover linhas onde todas as colunas são nulas (caso existam no futuro)
df_silver = df_silver.dropna(how="all")

# 5️⃣. Remover linhas se alguma coluna essencial estiver vazia
df_silver = df_silver.dropna(subset=["Home", "Away", "Res"])  # Ajuste conforme necessário

In [0]:
# 6️⃣. Criar `Match_ID` (Identificador único da partida)
df_silver = df_silver.withColumn("Match_ID", concat_ws("_", lit("Match"), monotonically_increasing_id()))

# 7️⃣. Criar a coluna Trimestre baseado na coluna Date
df_silver = df_silver.withColumn("Trimestre", quarter(col("Date")))

# 8️⃣. Criar a coluna Turno, para identificarmos os jogos do Primeiro e Segundo Turno
# Converter a coluna Date para o formato de data (se ainda não estiver)
df_silver = df_silver.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

# Criar a coluna 'Turno' com base na data
df_silver = df_silver.withColumn("Turno",
                                 when(col("Date") <= "2024-08-03", "Primeiro Turno")
                                 .otherwise("Segundo Turno"))

print("✅ Limpeza dos Dados e criação as colunas 'Match_ID', 'Trimestre' e 'Turno' concluída!")

✅ Limpeza dos Dados e criação as colunas 'Match_ID', 'Trimestre' e 'Turno' concluída!


## 🔶 Analise Geral do arquivo Tratado** (Checagem) 

Após a limpeza, vamos dar uma olhada nos campos desse arquivo.

- 1️⃣. Criar um DataFrame com a contagem de valores vazios por coluna
- 2️⃣. Exibir resumo estatístico das colunas + contagem de valores vazios

In [0]:
# 1️⃣. Criar um DataFrame com a contagem de valores vazios por coluna
empty_counts = df_silver.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_silver.columns])

In [0]:
# 2️⃣. Exibir resumo estatístico das colunas + contagem de valores vazios
print("📌 Estatísticas do DataFrame:")
df_silver.summary("count", "min", "max").show()

print("📌 Contagem de valores vazios por coluna:")
empty_counts.show()


📌 Estatísticas do DataFrame:
+-------+-------+-------+------+------------+------------+---+---+---+--------+---------+--------------+
|summary|Country| League|Season|        Home|        Away| HG| AG|Res|Match_ID|Trimestre|         Turno|
+-------+-------+-------+------+------------+------------+---+---+---+--------+---------+--------------+
|  count|    380|    380|   380|         380|         380|380|380|380|     380|      380|           380|
|    min| Brazil|Serie A|  2024|Athletico-PR|Athletico-PR|  0|  0|  A| Match_0|        2|Primeiro Turno|
|    max| Brazil|Serie A|  2024|     Vitoria|     Vitoria|  5|  6|  H|Match_99|        4| Segundo Turno|
+-------+-------+-------+------+------------+------------+---+---+---+--------+---------+--------------+

📌 Contagem de valores vazios por coluna:
+-------+------+------+----+----+----+----+---+---+---+--------+---------+-----+
|Country|League|Season|Date|Time|Home|Away| HG| AG|Res|Match_ID|Trimestre|Turno|
+-------+------+------+----+----

## 🔶 Unificação das Colunas `Home` e `Away` em uma só (Particularidade)

Essa é uma particularidade da nossa tabela, onde precisamos unificar as Colunas `Home` e `Away`, assim como as colunas relacionadas a ela também.

A definição dessa padronização foi feita durante o Diagrama Relacional, onde identificamos que as colunas separadas inviabilizariam nossas análises

- 1️⃣. Criação do Dataframe cuja coluna fosse `Home`
- 2️⃣. Criação do Dataframe cuja coluna fosse `Away`
- 3️⃣. Unificação dos dois Dataframes

In [0]:
from pyspark.sql.functions import col, lit

# 1️⃣. Criar DataFrame para os times que jogaram em casa (Home)
df_home = df_silver.withColumn("Clube", col("Home")) \
                   .withColumn("HomeOrAway", lit("Home")) \
                   .withColumn("Goals", col("HG")) \
                   .select("Country", "League", "Season", "Date", "Time", "Trimestre", "Turno", "Clube", "HomeOrAway", "Goals", "Res", "Match_ID")

In [0]:
# 2️⃣. Criar DataFrame para os times que jogaram fora (Away)
df_away = df_silver.withColumn("Clube", col("Away")) \
                   .withColumn("HomeOrAway", lit("Away")) \
                   .withColumn("Goals", col("AG")) \
                   .select("Country", "League", "Season", "Date", "Time", "Trimestre", "Turno", "Clube", "HomeOrAway", "Goals", "Res", "Match_ID")

In [0]:
# 3️⃣. Unir os dois DataFrames
df_silver = df_home.union(df_away)


## 🔶 Ajustes Finais

In [0]:
# Renomear colunas para um formato mais claro
df_silver = df_silver.withColumnRenamed("Country", "Pais") \
                     .withColumnRenamed("League", "Liga") \
                     .withColumnRenamed("Season", "Temporada") \
                     .withColumnRenamed("Date", "Data") \
                     .withColumnRenamed("Time", "Horario") \
                     .withColumnRenamed("HomeOrAway", "MandanteVisitante") \
                     .withColumnRenamed("Res", "Resultado")

print("✅ Colunas renomeadas para um formato mais legível.")

✅ Colunas renomeadas para um formato mais legível.


In [0]:
from pyspark.sql.functions import when

# 1️⃣. Substituir "Home" por "Mandante" e "Away" por "Visitante" na coluna MandanteVisitante
df_silver = df_silver.withColumn("MandanteVisitante",
                                 when(col("MandanteVisitante") == "Home", "Mandante")
                                 .when(col("MandanteVisitante") == "Away", "Visitante")
                                 .otherwise(col("MandanteVisitante")))

In [0]:
# 2️⃣. Atualizar a coluna Resultado conforme as regras
df_silver = df_silver.withColumn("Resultado",
                                 when(col("Resultado") == "D", "Empate")
                                 .when((col("MandanteVisitante") == "Mandante") & (col("Resultado") == "H"), "Vitoria")
                                 .when((col("MandanteVisitante") == "Visitante") & (col("Resultado") == "A"), "Vitoria")
                                 .otherwise("Derrota"))

print("✅ Transformações nas colunas MandanteVisitante e Resultado concluídas!")

✅ Transformações nas colunas MandanteVisitante e Resultado concluídas!


In [0]:
# Dicionário de padronização dos nomes dos clubes (em ordem alfabética)
mapeamento_clubes = {
    "Atletico GO": "Atlético-GO",
    "Atletico-MG": "Atlético-MG",
    "Athletico-PR": "Athletico-PR",
    "Bahia": "Bahia",
    "Botafogo RJ": "Botafogo",
    "Bragantino": "Bragantino",
    "Corinthians": "Corinthians",
    "Criciuma": "Criciúma",
    "Cruzeiro": "Cruzeiro",
    "Cuiaba": "Cuiabá",
    "Flamengo RJ": "Flamengo",
    "Fluminense": "Fluminense",
    "Fortaleza": "Fortaleza",
    "Gremio": "Grêmio",
    "Internacional": "Internacional",
    "Juventude": "Juventude",
    "Palmeiras": "Palmeiras",
    "Sao Paulo": "São Paulo",
    "Vasco": "Vasco da Gama",
    "Vitoria": "EC Vitória"
}

df_silver = df_silver.replace(mapeamento_clubes, subset=["Clube"])

## 💾 Salvar os dados na camada Silver (Parquet no S3)

In [0]:
# Caminho correto no S3 para armazenar os arquivos
silver_parquet_path = "s3://mvp-brasileirao-2024/silver/parquet/brasil_seriea_2024_todas_partidas"
silver_delta_path = "s3://mvp-brasileirao-2024/silver/delta/brasil_seriea_2024_todas_partidas"

# Remover o Delta anterior (use com cautela)
dbutils.fs.rm(silver_delta_path, recurse=True)

# Salvar em Parquet (para acessibilidade geral)
df_silver.write.mode("overwrite").parquet(silver_parquet_path)

# Salvar em Delta (para controle e governança)
df_silver.write.format("delta").mode("overwrite").save(silver_delta_path)

print("✅ Dados salvos corretamente em Parquet e Delta, em pastas separadas!")

✅ Dados salvos corretamente em Parquet e Delta, em pastas separadas!


In [0]:
# Caminho no DBFS para armazenar os dados Silver temporariamente
dbfs_path_silver = "dbfs:/mnt/silver_temp/todas_partidas"

# Salvar no DBFS em formato Parquet
df_silver.write.mode("overwrite").parquet(dbfs_path_silver)

print("✅ Dados transformados e salvos na camada Silver no DBFS com sucesso!")

✅ Dados transformados e salvos na camada Silver no DBFS com sucesso!
